In [57]:
import tensorflow as tf
print(tf.__version__)

from tensorflow import feature_column

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow.keras.utils as ku
from tensorflow.keras.utils import plot_model

import pandas as pd
from sklearn.model_selection import train_test_split

2.0.0-beta1


In [58]:
DATA_PATH = 'C:\SoloLearnMachineLearning\Stackoverflow\TextDataset.csv'

#it is just two column csv, like:
# text;label
# A wiki is run using wiki software;0
# otherwise known as a wiki engine.;1

dataframe = pd.read_csv(DATA_PATH, delimiter = ';')
dataframe.head()

,text,label
0,A wiki is run using wiki software,0
1,otherwise known as a wiki engine.,1
2,A wiki engine is a type of content management ...,1
3,"but it differs from most other such systems,in...",0
4,in that the content is created without any def...,0


In [59]:
# Preprocessing before feature_clolumn includes
# - getting the vocabulary
# - tokenization, which means only splitting on tokens. Encoding sentences with vocablary will be done by feature_column!
# - padding
# - truncating

# Build vacabulary
vocab_size = 100
oov_tok = '<OOV>'

sentences = dataframe['text'].to_list()

tokenizer = Tokenizer(num_words = vocab_size, oov_token="<OOV>")

tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index

# if word_index shorter then default value of vocab_size we'll save actual size
vocab_size=len(word_index)
print("vocab_size = word_index = ",len(word_index))

vocab_size = word_index =  59


In [60]:
# Split sentensec on tokens. here token = word
# text_to_word_sequence() has good default filter for charachters include basic punctuation, tabs, and newlines
dataframe['text'] = dataframe['text'].apply(tf.keras.preprocessing.text.text_to_word_sequence)

In [61]:
dataframe.head()

,text,label
0,"[a, wiki, is, run, using, wiki, software]",0
1,"[otherwise, known, as, a, wiki, engine]",1
2,"[a, wiki, engine, is, a, type, of, content, ma...",1
3,"[but, it, differs, from, most, other, such, sy...",0
4,"[in, that, the, content, is, created, without,...",0


In [62]:
max_length = 6

# paddind and trancating setnences
# do that directly with strings without using tokenizer.texts_to_sequences()
# the feature_colunm will convert strings into numbers
dataframe['text']=dataframe['text'].apply(lambda x, N=max_length: (x + N * [''])[:N])
dataframe['text']=dataframe['text'].apply(lambda x, N=max_length: x[:N])
dataframe.head()

,text,label
0,"[a, wiki, is, run, using, wiki]",0
1,"[otherwise, known, as, a, wiki, engine]",1
2,"[a, wiki, engine, is, a, type]",1
3,"[but, it, differs, from, most, other]",0
4,"[in, that, the, content, is, created]",0


In [63]:
# Define method to create tf.data dataset from Pandas Dataframe
def df_to_dataset(dataframe, label_column, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    #labels = dataframe.pop(label_column)
    labels = dataframe[label_column]

    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds

In [64]:
# Split dataframe into train and validation sets
train_df, val_df = train_test_split(dataframe, test_size=0.2)

print(len(train_df), 'train examples')
print(len(val_df), 'validation examples')

7 train examples
2 validation examples


In [65]:
batch_size = 32
ds = df_to_dataset(dataframe, 'label',shuffle=False,batch_size=batch_size)

train_ds = df_to_dataset(train_df, 'label',  shuffle=False, batch_size=batch_size)
val_ds = df_to_dataset(val_df, 'label', shuffle=False, batch_size=batch_size)

In [66]:
# and small batch for demo
example_batch = next(iter(ds))[0]
example_batch

{'text': <tf.Tensor: id=3387, shape=(9, 6), dtype=string, numpy=
 array([[b'a', b'wiki', b'is', b'run', b'using', b'wiki'],
        [b'otherwise', b'known', b'as', b'a', b'wiki', b'engine'],
        [b'a', b'wiki', b'engine', b'is', b'a', b'type'],
        [b'but', b'it', b'differs', b'from', b'most', b'other'],
        [b'in', b'that', b'the', b'content', b'is', b'created'],
        [b'and', b'wikis', b'have', b'little', b'inherent', b'structure'],
        [b'allowing', b'structure', b'to', b'emerge', b'according', b'to'],
        [b'there', b'are', b'dozens', b'of', b'different', b'wiki'],
        [b'both', b'standalone', b'and', b'part', b'of', b'other']],
       dtype=object)>,
 'label': <tf.Tensor: id=3386, shape=(9,), dtype=int32, numpy=array([0, 1, 1, 0, 0, 1, 0, 1, 0])>}

In [67]:
# Helper method to print exxample outputs of for defined feature_column

def demo(feature_column):
    feature_layer = tf.keras.layers.DenseFeatures(feature_column)
    print(feature_layer(example_batch).numpy())

In [68]:
# Define categorical colunm for our text feature, which is preprocessed into lists of tokens
# Note that key name should be the same as original column name in dataframe
text_column = feature_column.categorical_column_with_vocabulary_list(key='text', 
                                                                     vocabulary_list=list(word_index))

#indicator_column produce one-hot-encoding. These lines just to compare with embedding
#print(demo(feature_column.indicator_column(payment_description_3)))
#print(payment_description_2,'\n')

# arguemnt dimention here is exactly the dimension of the space in which tokens will be presented during model's learning
# see the tutorial at https://www.tensorflow.org/beta/tutorials/text/word_embeddings
text_embedding = feature_column.embedding_column(text_column, dimension=8)
print(demo(text_embedding))

[[-0.17686643 -0.2989895   0.00264741 -0.3476688  -0.05737228 -0.18041377
  -0.00098308  0.325977  ]
 [ 0.00710045 -0.00310291 -0.02391201 -0.14713128 -0.04860273 -0.14387627
   0.0257867  -0.02231171]
 [-0.10285576 -0.18304147 -0.00298556  0.01808989 -0.16199614 -0.08612611
   0.1993926   0.15643066]
 [ 0.12189802 -0.14372785 -0.0255752  -0.11260048 -0.17380564 -0.10022938
  -0.17018622 -0.33584765]
 [-0.14319299 -0.07304262 -0.02680805  0.10860274 -0.03133818  0.05705167
   0.09285476  0.02952158]
 [ 0.0521474  -0.04482817  0.0456489  -0.09259579  0.0093843  -0.09475025
  -0.12256879  0.24423139]
 [-0.11564245  0.11739489  0.07430411 -0.10169675 -0.24093457  0.06556444
  -0.00776027  0.20457251]
 [-0.09204625 -0.04545312 -0.07439894 -0.20941615 -0.09087479 -0.00044735
   0.03422147 -0.19390172]
 [-0.1145098   0.17188363 -0.12575074  0.07546506  0.00690869  0.01805285
   0.14318828 -0.04583522]]
None


In [69]:
# The define the layers and model it self
# This example uses Keras Functional API instead of Sequential just for more generallity

# Define DenseFeatures layer to pass feature_columns into Keras model
feature_layer = tf.keras.layers.DenseFeatures(text_embedding)

In [70]:
# Define inputs for each feature column. See
# см. https://github.com/tensorflow/tensorflow/issues/27416#issuecomment-502218673
feature_layer_inputs = {}

# Here we have just one column
feature_layer_inputs['text'] = tf.keras.Input(shape=(max_length,), name='text', dtype=tf.string)
print(feature_layer_inputs)

{'text': <tf.Tensor 'text_4:0' shape=(None, 6) dtype=string>}


In [71]:
# Define outputs of DenseFeatures layer 
# And accually use them as first layer of the model
feature_layer_outputs = feature_layer(feature_layer_inputs)
print(feature_layer_outputs)

# Add consequences layers. See https://keras.io/getting-started/functional-api-guide/
x = tf.keras.layers.Dense(256, activation='relu')(feature_layer_outputs)
x = tf.keras.layers.Dropout(0.2)(x)

# This example supposes binary classification, as labels are 0 or 1
x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(inputs=[v for v in feature_layer_inputs.values()], outputs=x)

model.summary()

# This example supposes binary classification, as labels are 0 or 1
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy']
              #run_eagerly=True
             )

Tensor("dense_features_6/Identity:0", shape=(None, 8), dtype=float32)
Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text (InputLayer)            [(None, 6)]               0         
_________________________________________________________________
dense_features_6 (DenseFeatu (None, 8)                 472       
_________________________________________________________________
dense_12 (Dense)             (None, 256)               2304      
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 257       
Total params: 3,033
Trainable params: 3,033
Non-trainable params: 0
_________________________________________________________________


In [72]:
# Note that fit() method looking up features in train_ds and valdation_ds by name in 
# tf.keras.Input(shape=(max_length,), name='text'

# This model of cause will learn nothing because of fake data.

num_epochs = 5
history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=num_epochs,
                    verbose=1
                    )

W0823 14:13:15.540783 18764 training_utils.py:1436] Expected a shuffled dataset but input dataset `x` is not shuffled. Please invoke `shuffle()` on input dataset.


Epoch 1/5
1/1 [==============================] - 1s 1s/step - loss: 0.6900 - accuracy: 0.7143 - val_loss: 0.7230 - val_accuracy: 0.0000e+00
Epoch 2/5
1/1 [==============================] - 0s 9ms/step - loss: 0.6867 - accuracy: 0.5714 - val_loss: 0.7294 - val_accuracy: 0.0000e+00
Epoch 3/5
1/1 [==============================] - 0s 10ms/step - loss: 0.6798 - accuracy: 0.8571 - val_loss: 0.7356 - val_accuracy: 0.0000e+00
Epoch 4/5
1/1 [==============================] - 0s 10ms/step - loss: 0.6763 - accuracy: 0.7143 - val_loss: 0.7420 - val_accuracy: 0.0000e+00
Epoch 5/5
1/1 [==============================] - 0s 10ms/step - loss: 0.6683 - accuracy: 0.8571 - val_loss: 0.7485 - val_accuracy: 0.0000e+00
